In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rasterio

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
import ee
import logging
from shapely.geometry import mapping

# Define paths
BASE_DIR = "/content/drive/MyDrive/wildlands_league"
OUTPUT_DIR = BASE_DIR
STUDY_AREA_PATH = os.path.join(BASE_DIR, "Algonquin_Park_Layers.gpkg")

# Confirm paths
print("Working directory:", OUTPUT_DIR)
print("Study area file:", STUDY_AREA_PATH)

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
import ee

# Authenticate with your Google account
try:
    ee.Initialize(project='ee-komba')
except ee.EEException:
    ee.Authenticate()
    ee.Initialize(project='ee-komba')

In [ ]:
# Manually uploaded local file paths
glad_datasets = {
    "Forest_extent_2000": os.path.join(OUTPUT_DIR, "Forest_extent_2000.tif"),
    "Forest_extent_2020": os.path.join(OUTPUT_DIR, "Forest_extent_2020.tif"),
    "Net_loss_2000_2020": os.path.join(OUTPUT_DIR, "netloss.tif"),
    "Forest_height_2000": os.path.join(OUTPUT_DIR, "Forest_Height_2000.tif"),
    "Forest_height_2020": os.path.join(OUTPUT_DIR, "forest_height_2020.tif"),
    "Net_gain_2000_2020": os.path.join(OUTPUT_DIR, "netgain.tif")
}

In [ ]:
# Read AOI
aoi = gpd.read_file(STUDY_AREA_PATH)
aoi = aoi.to_crs("EPSG:4326")

# Initialize stats_list
stats_list = []

# Loop through datasets and process
for name, local_tif in glad_datasets.items():
    if not os.path.exists(local_tif):
        print(f"Missing: {local_tif}")
        continue

    print(f"Processing {name}...")
    with rasterio.open(local_tif) as src:
        out_image, out_transform = rasterio.mask.mask(src, aoi.geometry.map(mapping), crop=True)
        out_meta = src.meta.copy()

        # Update metadata
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Save clipped raster
        clipped_path = os.path.join(OUTPUT_DIR, f"{name}_clipped.tif")
        with rasterio.open(clipped_path, "w", **out_meta) as dest:
            dest.write(out_image)

        # Calculate statistics
        data = out_image[0]
        data = data[data != src.nodata] if src.nodata is not None else data
        
        # Add stats to stats_list
        stats = {
            "Dataset": name,
            "Min": float(np.min(data)),
            "Max": float(np.max(data)),
            "Mean": float(np.mean(data)),
            "Non_zero_pixels": int((data > 0).sum())
        }
        stats_list.append(stats)
        
        # Print basic stats
        print(f"  Min: {stats['Min']}")
        print(f"  Max: {stats['Max']}")
        print(f"  Mean: {stats['Mean']:.2f}")
        print(f"  Non-zero pixels: {stats['Non_zero_pixels']}")

print("\nCompleted processing all datasets")

In [ ]:
# Create stats_df from stats_list
if len(stats_list) > 0:
    stats_df = pd.DataFrame(stats_list)
    print("✅ Created stats_df from stats_list")
    
    # Calculate areas
    PIXEL_AREA_HA = 0.09  # 30m x 30m = 900m² = 0.09 ha
    stats_df["Area_ha"] = stats_df["Non_zero_pixels"] * PIXEL_AREA_HA
    
    # Save to CSV
    csv_path = os.path.join(OUTPUT_DIR, "forest_stats_with_area.csv")
    stats_df.to_csv(csv_path, index=False)
    print(f"✅ Saved stats with area to {csv_path}")
    
    # Display the results
    print("\nForest Analysis Results:")
    print(stats_df.to_string(index=False))
else:
    raise ValueError("❌ No data was processed. Please check the input datasets.")

In [ ]:
# Create a buffer around the AOI
buffer_distance_m = 5000  # 5 km buffer

# Project AOI to UTM Zone 17N for Ontario
aoi_projected = aoi.to_crs(epsg=32617)
aoi_buffered = aoi_projected.buffer(buffer_distance_m)
aoi_buffered_gdf = gpd.GeoDataFrame(geometry=aoi_buffered, crs=aoi_projected.crs)

# Reproject buffer back to WGS84
aoi_buffered_wgs84 = aoi_buffered_gdf.to_crs("EPSG:4326")
buffer_geom = [aoi_buffered_wgs84.unary_union]

# Analyze datasets using the buffer zone
buffer_stats_list = []

for name, local_tif in glad_datasets.items():
    if not os.path.exists(local_tif):
        print(f"Missing: {local_tif}")
        continue

    with rasterio.open(local_tif) as src:
        try:
            out_image, out_transform = mask(src, buffer_geom, crop=True)
            
            data = out_image[0]
            data = data[data != src.nodata] if src.nodata is not None else data
            
            if data.size > 0:
                stats = {
                    "Dataset": name,
                    "Region": "Buffer Zone",
                    "Min": float(np.min(data)),
                    "Max": float(np.max(data)),
                    "Mean": float(np.mean(data)),
                    "Non_zero_pixels": int((data > 0).sum()),
                    "Area_ha": int((data > 0).sum()) * PIXEL_AREA_HA
                }
                buffer_stats_list.append(stats)
                print(f"Processed {name} for buffer zone")
                
        except Exception as e:
            print(f"Error processing {name} for buffer zone: {e}")

# Combine core and buffer statistics
if 'stats_df' in globals():
    stats_df["Region"] = "Core AOI"
    combined_df = pd.concat([stats_df, pd.DataFrame(buffer_stats_list)], ignore_index=True)
    
    # Save combined results
    combined_csv_path = os.path.join(OUTPUT_DIR, "forest_extent_height_with_buffer.csv")
    combined_df.to_csv(combined_csv_path, index=False)
    print(f"\nSaved combined analysis to {combined_csv_path}")
    
    # Display results
    print("\nCombined Analysis Results:")
    print(combined_df.to_string(index=False))
else:
    print("❌ Core AOI analysis not found. Only buffer zone results available.")